In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# 导入Fmnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'FMNIST'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
# direct use real_label,normalize var as 0.5
fake_noise_normal = torch.from_numpy(np.random.normal(loc=0.5, scale=0.5, size=[2048,1,32,32])) # var is 1/4
fake_noise_normal = fake_noise_normal.type(torch.FloatTensor)
fake_noise_normal = fake_noise_normal.to(device)
fake_label = torch.from_numpy(np.random.randint(10,size=[2048]))

fake_noise_uniform = np.sqrt(3)*torch.from_numpy(np.random.uniform(0,1,[2048,1,32,32])) # 1/12
fake_noise_uniform = fake_noise_uniform.type(torch.FloatTensor)
fake_noise_uniform = fake_noise_uniform.to(device)

fake_noise_cauchy = torch.from_numpy(np.random.standard_cauchy(size=[2048,1,32,32])+0.5)
fake_noise_cauchy = fake_noise_cauchy.type(torch.FloatTensor)
fake_noise_cauchy = fake_noise_cauchy.to(device)

fake_noise_laplace = torch.from_numpy(np.random.laplace(loc=0.5, scale=1.0, size=[2048,1,32,32]))/ np.sqrt(8) # var is 2
fake_noise_laplace = fake_noise_laplace.type(torch.FloatTensor)
fake_noise_laplace = fake_noise_laplace.to(device)

fake_noise_gumbel = torch.from_numpy(np.random.gumbel(loc=0.5,scale=1.0,size=[2048,1,32,32]))/ np.sqrt(6.58) #pi^2/6 1.645
fake_noise_gumbel = fake_noise_gumbel.type(torch.FloatTensor)
fake_noise_gumbel = fake_noise_gumbel.to(device)
# from 0.01 - 0.99,0 -1
epsilon_noise = 0.1

In [4]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [5]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10



In [6]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius_noise(radius1,radius2,netD,noise, real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [7]:
# load_model
netD = torch.load('FMNIST_NetD_changing2-mode11.pth') 
netV = torch.load('FMNIST_NetV_changing2-mode11.pth') 
netG = torch.load('FMNIST_NetG_changing2-mode11.pth')

mode = 11
gam3 = 1
net_noise1 = get_noise(DATASET, device, LABEL)

In [8]:
# data_preprocessing and parameters

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET2 = 'mnist'
ROOT2    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader2, dataset2 = get_dataloader(
        data_name=DATASET2,
        root=ROOT2,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader2 = iter(dataloader2)
data2, label2 = next(iter_dataloader2)
data_time = time.time() - data_time
real_cpu2 = data2.to(device)
real_label2 = label2.clone().detach()

In [9]:
# data_preprocessing and parameters
import time
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET3 = 'cifar10'
ROOT3    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader3, dataset3 = get_dataloader(
        data_name=DATASET3,
        root=ROOT3,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader3 = iter(dataloader3)
data3, label3 = next(iter_dataloader3)
data_time = time.time() - data_time
real_cpu3 = data3.to(device)
real_label3 = label3.clone().detach()

sample1 = real_cpu3[:,0,:,:]
sample2 = real_cpu3[:,1,:,:]
sample3 = real_cpu3[:,2,:,:]

cifar_sample1 = torch.unsqueeze(sample1, dim=1)
cifar_sample2 = torch.unsqueeze(sample2, dim=1)
cifar_sample3 = torch.unsqueeze(sample3, dim=1)

Files already downloaded and verified


In [10]:
#sample the length of radius,gaussian noise
mode = 11
gam3 = 1

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#sample the length of radius,gaussian noise
mode = 1
gam3 = 1

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)


mode = 1
gam3 = 0.5

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)



mode = 1
gam3 = 2

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.5842790683940798
1.7252797384709477
2.3899288501800573
2.5044017193826695
2.3366347792155113
5.564193263410672
4.363148720645572
99.97706794914475
99.92577785517669
99.97825883872153
4.325044903575554
3.872243924829366
99.97706763893318
99.80781682417752
99.91508287393698
7.162348816513137
26.002503588822265
99.97706794914475
99.9807818504967
99.97825883872153


In [11]:
#sample the length of radius,gaussian noise
net_noise2 = get_noise(DATASET, device, LABEL)
mode = 11
gam3 = 1

r1,r2 = gs_radius_noise(0,100,netD,net_noise2,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 1

r1,r2 = gs_radius_noise(0,100,netD,net_noise2,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)


mode = 1
gam3 = 0.5

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)


mode = 1
gam3 = 2

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.5842790683940798
1.7252797384709477
2.3899288501800573
2.5044017193826695
2.3366347792155113
5.564193263410672
4.363148720645572
99.97706794914475
99.92577785517669
99.97825883872153
4.325044903575554
3.872243924829366
99.97706763893318
99.80781682417752
99.91508287393698
7.162348816513137
26.002503588822265
99.97706794914475
99.9807818504967
99.97825883872153


In [12]:
#sample the length of radius,gaussian noise
net_noise3 = get_noise(DATASET, device, LABEL)
mode = 11
gam3 = 1

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 1

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 0.1

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 0.2

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 0.3

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 0.4

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 0.5

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 0.8

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 1.5

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 2

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.5842790683940798
1.7252797384709477
2.3899288501800573
2.5044017193826695
2.3366347792155113
5.564193263410672
4.363148720645572
99.97706794914475
99.92577785517669
99.97825883872153
0.06101165494517871
2.6220183207272583
4.060964654216485
4.095554782168584
3.92629352339857
0.9293298009406953
3.16567551686633
6.7792227357569255
6.17107291930214
6.276454036371153
2.595593042524447
3.482762717341521
12.38422984502941
11.384842960881642
14.393303607156227
3.7362692603431515
3.7010794930391318
99.4973979540648
69.00145563485194
99.82705024528919
4.325044903575554
3.872243924829366
99.97706763893318
99.80781682417752
99.91508287393698
5.159298103214493
4.212498656577533
99.97706794914475
99.80781682417752
99.91508287393698
6.44379517285175
4.598176295612283
99.97706794914475
99.9807818504967
99.97825883872153
7.162348816513137
26.002503588822265
99.97706794914475
99.9807818504967
99.97825883872153


In [13]:
# sample 20 times
mode = 11
gam3 = 1

L_mnist = []
L_Fmnist = []
L_cifar1 = []
L_cifar2 = []
L_cifar3 = []

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_mnist.append(r1)
print(max(L_mnist),min(L_mnist),np.mean(L_mnist),np.var(L_mnist))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_Fmnist.append(r1)
print(max(L_Fmnist),min(L_Fmnist),np.mean(L_Fmnist),np.var(L_Fmnist))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_cifar1.append(r1)
print(max(L_cifar1),min(L_cifar1),np.mean(L_cifar1),np.var(L_cifar1))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_cifar2.append(r1)
print(max(L_cifar2),min(L_cifar2),np.mean(L_cifar2),np.var(L_cifar2))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_cifar3.append(r1)
print(max(L_cifar3),min(L_cifar3),np.mean(L_cifar3),np.var(L_cifar3))


1.478902961965466
1.796028536013361
1.6616136402789108
1.6289411973919274
1.7160215562399146
1.6377002368741393
1.7482152301335974
1.5119341788520722
1.4789036556366695
1.9039020401041196
1.8302507612009384
1.5034013809856266
1.5996255066244809
1.7355988114639664
1.6493548000389833
1.7315168886233017
1.580170363457247
1.5465293730016938
1.8551188267621217
1.8208937807906713
1.9039020401041196 1.478902961965466 1.67073118632196 0.016344619952380672
0.5337096984880556
0.5572418899092717
0.5424712933485455
0.5632693867875774
0.5669830963906498
0.5648287997753794
0.5368280225413506
0.5912524532345743
0.5109966232133467
0.5302225407123126
0.5490768484492375
0.537423408091128
0.538984947806553
0.5541234925843228
0.5028297201461247
0.5160427652880878
0.5720302424861929
0.5368278308306048
0.5216834074453507
0.5862065028705792
0.5912524532345743 0.5028297201461247 0.5456516485199623 0.0005457737099566115
2.503438312344195
2.4144238733077503
2.5156848931852527
2.599079388066439
2.487104934784068

In [14]:
#sample the length of radius,gaussian noise
mode = 11
gam3 = 1
L_noise= []
epsilon = 0.05
L_noise1= []
epsilon1 = 0.1

L_noise2= []
epsilon2 = 0.2

L_noise3= []
epsilon3 = 0.3
L_noise4= []
epsilon4 = 0.4
L_noise5= []
epsilon5 = 0.5
L_noise6= []
epsilon6 = 0.6
L_noise7= []
epsilon7 = 0.7
L_noise8= []
epsilon8 = 0.8
L_noise9= []
epsilon9 = 0.9

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise.append(r1)
print(max(L_noise),min(L_noise),np.mean(L_noise),np.var(L_noise))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise1.append(r1)
print(max(L_noise1),min(L_noise1),np.mean(L_noise1),np.var(L_noise1))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise2.append(r1)
print(max(L_noise2),min(L_noise2),np.mean(L_noise2),np.var(L_noise2))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise3.append(r1)
print(max(L_noise3),min(L_noise3),np.mean(L_noise3),np.var(L_noise3))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise4.append(r1)
print(max(L_noise4),min(L_noise4),np.mean(L_noise4),np.var(L_noise4))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise5.append(r1)
print(max(L_noise5),min(L_noise5),np.mean(L_noise5),np.var(L_noise5))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise6.append(r1)
print(max(L_noise6),min(L_noise6),np.mean(L_noise6),np.var(L_noise6))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise7.append(r1)
print(max(L_noise7),min(L_noise7),np.mean(L_noise7),np.var(L_noise7))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise8.append(r1)
print(max(L_noise8),min(L_noise8),np.mean(L_noise8),np.var(L_noise8))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise9.append(r1)
print(max(L_noise9),min(L_noise9),np.mean(L_noise9),np.var(L_noise9))

0.5449944969197689
0.4950327734082927
0.5481128209730639
0.5443991113699915
0.5349008982764143
0.5638652010261588
0.5475172437125406
0.5296271551625352
0.5654241853251567
0.5119600302518215
0.5424712933485455
0.5475174354232866
0.5550874015832554
0.5393531610059963
0.5302225407123126
0.5393529692952505
0.5562782911600511
0.5735896555121439
0.568911032827607
0.5308180447393309
0.5735896555121439 0.4950327734082927 0.5434717871016762 0.0003391735718467639
0.5591691326750252
0.5308183549508938
0.534305084037833
0.5383869336068438
0.5679471971003284
0.5572416981985259
0.538984947806553
0.5623059797491027
0.5154463757938139
0.49466475191959536
0.5525643898080838
0.5415074576212667
0.5515992397867102
0.5617104757220843
0.5459583326470475
0.5098031049866192
0.552562765302426
0.5576099113979691
0.5490767299719965
0.5242064192205224
0.5679471971003284 0.49466475191959536 0.5422934641151621 0.00035649004428661777
0.5135191330280605
0.49154635459464635
0.5550868996227976
0.5735896555121439
0.5129


# other data sets, using same noise to get different r for diff data
import time
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET2 = 'FMNIST'
ROOT2    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader2, dataset2 = get_dataloader(
        data_name=DATASET2,
        root=ROOT2,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader2 = iter(dataloader2)
data2, label2 = next(iter_dataloader2)
data_time = time.time() - data_time
real_cpu2 = data2.to(device)
real_label2 = label2.clone().detach()


device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET3 = 'cifar10'
ROOT3    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader3, dataset3 = get_dataloader(
        data_name=DATASET3,
        root=ROOT3,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader3 = iter(dataloader3)
data3, label3 = next(iter_dataloader3)
data_time = time.time() - data_time
real_cpu3 = data3.to(device)
real_label3 = label3.clone().detach()

sample1 = real_cpu2[:,0,:,:]
sample2 = real_cpu2[:,1,:,:]
sample3 = real_cpu2[:,2,:,:]